## Практическое задание

<ol>
    <li>Попробуйте обучить нейронную сеть U-Net на любом другом датасете. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?
    </li>
    <li>*Попробуйте свои силы в задаче Carvana на Kaggle - https://www.kaggle.com/c/carvana-image-masking-challenge/overview</li>
    <li>*Сделайте свою реализацию U-Net на TensorFlow</li>
</ol>

## Подключение библиотек и настройка параметров проекта

In [0]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH = '/content/drive/My Drive/GU_neural_network/Carvana'
BATCH_SIZE = 32
RANDOM_STATE = 1
SHAPE = (128, 128)

In [0]:
import sys
import os
sys.path.append(PATH)

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random

In [0]:
import tensorflow as tf
from skimage import io, transform
from tensorflow.keras.preprocessing.image import array_to_img ,img_to_array ,ImageDataGenerator ,load_img
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, concatenate, Input, Flatten
from tensorflow.keras import Model

In [0]:
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

## Прототип Unet 

In [0]:
class DownLayer(layers.Layer):
    downconv: Conv2D = None
    residual: Conv2D = None
    max_pool: MaxPool2D = None

    def __init__(self, filters, pool=True, **kwargs):
        super(DownLayer, self).__init__(**kwargs)
        self.downconv = Conv2D(filters, (3, 3), padding='same', activation='relu')
        self.residual = Conv2D(filters, (3, 3), padding='same', activation='relu')
        if pool:
            self.max_pool = MaxPool2D()

    def call(self, inputs):
        x = self.downconv(inputs)
        x_residual = self.residual(x)
        if self.max_pool is None:
            return None, x_residual
        else:
            x_max_pool = self.max_pool(x_residual)
            return x_max_pool, x_residual

    def get_config(self):
        config = super(DownLayer, self).get_config()
        config.update({'conv': self.downconv.get_config()})
        config.update({'residual': self.residual.get_config()})
        if self.max_pool is None:
            config.update({'max_pool': self.max_pool})
        else:
            config.update({'max_pool': self.max_pool.get_config()})
        return config

class UpLayer(layers.Layer):
    upconv: Conv2D = None
    upconv1: Conv2D = None
    upconv2: Conv2D = None
    upsample: UpSampling2D = None
    concat_inputs: concatenate = None

    def __init__(self, filters, **kwargs):
        super(UpLayer, self).__init__(**kwargs)
        filters = int(filters)
        self.upsample = UpSampling2D()
        self.upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")
        self.upconv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')
        self.upconv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')


    def call(self, inputs, residual):
        x = self.upsample(inputs)
        x = self.upconv(x)
        x = concatenate(inputs=[residual, x], axis=3)
        x = self.upconv1(x)
        return self.upconv2(x)

    def get_config(self):
        config = super(UpLayer, self).get_config()
        config.update({'upconv': self.upconv.get_config()})
        config.update({'upsample': self.upsample.get_config()})
        config.update({'upconv1': self.upconv1.get_config()})
        config.update({'upconv2': self.upconv2.get_config()})
        return config

class UNet(Model):
    downcascade: [] = []
    upcascade: [] = []
    residuals: [] = []
    count_layers: int = 4
    input_layer: Input = None
    output_layer: Conv2D = None

    def __init__(self, input_shape=[128, 128, 3], pool=True, **kwargs):
        super(UNet, self).__init__()
        filters = 64
        #self.input_layer = Input(shape=input_shape)
        self.output_layer = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")
        for i in range(self.count_layers):
            #print(f'{i}.{filters * (2 ** i)}')
            self.downcascade.append(DownLayer(filters=filters * (2 ** i), pool=pool))
            self.upcascade.append(UpLayer(filters=filters * (2 ** i)))

    def call(self, inputs):
        x = inputs#self.input_layer(inputs)
        for i in range(self.count_layers):
            x, res = self.downcascade[i](x)
            self.residuals.append(res)

        for i in range(self.count_layers):
            x = self.upcascade[i](inputs=x, residual=self.residuals[-i-1])
        return self.output_layer(x)

Замечание: Полноценный класс реализовать не удалось. Есть ошибка в коде, но провести трасировку и найти не удалось. Model.summary() не выводит корректно слои.

## Использование готовой нейронной сети

In [0]:
!pip install git+https://github.com/tensorflow/examples.git

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-kgpezz_7
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-req-build-kgpezz_7
  Created wheel for tensorflow-examples: filename=tensorflow_examples-ce15879e289a17da6ba6f8e36faca0a98cec56bd_-cp36-none-any.whl size=115891 sha256=7ee5dedaea286d16a9312e0fd6bf9b4f9b8c3bcc74d82fe4ada710ad7b48f9ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-umywlhd1/wheels/83/64/b3/4cfa02dc6f9d16bf7257892c6a7ec602cd7e0ff6ec4d7d714d
Successfully built tensorflow-examples


In [0]:
from tensorflow_examples.models.pix2pix import pix2pix

def basemodel(name='basemodel', input_shape=[128, 128, 3], output_channels=1):
  base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

  # Use the activations of these layers
  layer_names = [
      'block_1_expand_relu',   # 64x64
      'block_3_expand_relu',   # 32x32
      'block_6_expand_relu',   # 16x16
      'block_13_expand_relu',  # 8x8
      'block_16_project',      # 4x4
  ]
  layers = [base_model.get_layer(name).output for name in layer_names]

  # Create the feature extraction model
  down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

  down_stack.trainable = False

  up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
  ]

  inputs = tf.keras.layers.Input(shape=input_shape)
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(name=name, inputs=inputs, outputs=x)

## Генератор данных из загруженных файлов. 

За основу взят класс https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c. Использовать tensorflow-dataset.carvana не удалось (взят был из ветки гитхаба)

In [0]:
import cv2
from tensorflow.keras.utils import Sequence

#base class copy from:https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, labels, image_path, mask_path,
                 to_fit=True, batch_size=32, dim=(256, 256),
                 n_channels=3, n_classes=10, shuffle=True):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.list_IDs = list_IDs
        self.labels = labels
        self.image_path = image_path
        self.mask_path = mask_path
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self._generate_X(list_IDs_temp)

        if self.to_fit:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X

    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def _generate_X(self, list_IDs_temp):
        """Generates data containing batch_size images
        :param list_IDs_temp: list of label ids to load
        :return: batch of images
        """
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self._load_grayscale_image(self.image_path + self.labels[ID], False)

        return X

    def _generate_y(self, list_IDs_temp):
        """Generates data containing batch_size masks
        :param list_IDs_temp: list of label ids to load
        :return: batch if masks
        """
        y = np.empty((self.batch_size, *self.dim), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            y[i,] = self._load_grayscale_image(self.mask_path + self.labels[ID].split('.')[0] +'_mask.gif', True)

        return y

    def _load_grayscale_image(self, image_path, is_mask=True):
        """Load grayscale image
        :param image_path: path to image to load
        :return: loaded image
        """
        img = io.imread(image_path, as_gray=is_mask)
        img  = transform.resize(img, output_shape=self.dim, mode='constant')
        img = img / 255
        return img

## Обучение

### Подготовка датасетов

In [0]:
import pandas as pd
all_imgs = pd.read_csv(os.path.join(PATH,'Source/train_masks.csv'))  

In [0]:
image_path = os.path.join(PATH,'Source/train/')  
mask_path = os.path.join(PATH,'Source/train_masks/')  

train_idx ,val_idx = train_test_split(all_imgs.index ,train_size =0.8 ,test_size =0.2, random_state=RANDOM_STATE)

if False:
  train_idx = train_idx[:100]
  val_idx = val_idx[:100]

training_generator = DataGenerator(train_idx, all_imgs['img'], image_path, mask_path, batch_size=BATCH_SIZE, dim=SHAPE )
validation_generator = DataGenerator(val_idx, all_imgs['img'], image_path, mask_path, batch_size=BATCH_SIZE, dim=SHAPE )

### Loss-функция

In [0]:
def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
  denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

  return 1 - (numerator + 1) / (denominator + 1)

### Обучение или загрузка готовой модели

In [0]:
USE_TRAIN_MODEL = True

In [0]:
if USE_TRAIN_MODEL:
  # Восстановим в точности ту же модель, включая веса и оптимизатор
  model = tf.keras.models.load_model(os.path.join(PATH,'basemodel.h5'), custom_objects={"dice_loss":dice_loss})
else:
  model = basemodel()
model.compile(optimizer='adam',loss=dice_loss, metrics=['accuracy', dice_loss])

In [0]:
callbacks = [
  # Остановить обучение если `val_loss` перестанет улучшаться в течение 2 эпох
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

In [0]:
if not USE_TRAIN_MODEL:
  history = model.fit(training_generator, epochs=3, validation_data=validation_generator, batch_size=BATCH_SIZE, verbose=1, callbacks=callbacks)
  model.save(os.path.join(PATH,'basemodel.h5'))

## Предсказание 

In [0]:
test_path = os.path.join(PATH,'Source/test/')
all_test_imgs = os.listdir(test_path)
test_idx = range(len(all_test_imgs))
test_generator = DataGenerator(test_idx, all_test_imgs , test_path, mask_path=None, to_fit=False)

In [0]:
pred = model.predict(test_generator, verbose=1)

562/562 [==============================] - 10339s 18s/step


Сохранение результатов

In [0]:
np.save(os.path.join(PATH , 'test_pred.npy'), pred)

### TODO: преобразование в rle и подготовка финального submit